## Preprocessing

In [225]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [226]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df=application_df.drop(['NAME','EIN'], axis = 1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [227]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [228]:
# Look at APPLICATION_TYPE value counts for binning
value_counts = application_df['APPLICATION_TYPE'].value_counts()
value_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [229]:

# Replace values if count is less than 200
replace_values = value_counts[value_counts < 200].index # index retrieves the indices (unique values) from the value_counts 
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(replace_values, 'Other')

# Print updated value counts
updated_value_counts = application_df['APPLICATION_TYPE'].value_counts()
print(updated_value_counts)

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64


In [230]:
# Look at CLASSIFICATION value counts for binning
class_count=application_df['CLASSIFICATION'].value_counts()
class_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [231]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_cutoff=class_count[class_count>1]
class_cutoff

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [232]:
replace_value_class = class_count[class_count<1883].index
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(replace_value_class, 'Other')

# Print updated value counts
updated_value_counts = application_df['CLASSIFICATION'].value_counts()
print(updated_value_counts)

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64


In [233]:
# dropping potential variables
application_df = application_df.drop(['SPECIAL_CONSIDERATIONS'], axis =1)

In [234]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)


In [235]:
# Split our preprocessed data into our features and target arrays
y= application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'], axis =1).values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [236]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [237]:
number_input_features = len(X_train[0])
number_input_features

41

## Compile, Train and Evaluate the Model

In [238]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_46 (Dense)            (None, 10)                420       
                                                                 
 dense_47 (Dense)            (None, 5)                 55        
                                                                 
 dense_48 (Dense)            (None, 1)                 6         
                                                                 
Total params: 481
Trainable params: 481
Non-trainable params: 0
_________________________________________________________________


In [239]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [240]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50


804/804 [==============================] - 2s 1ms/step - loss: 0.6228 - accuracy: 0.6819
Epoch 2/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5653 - accuracy: 0.7232
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5585 - accuracy: 0.7248
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5554 - accuracy: 0.7259
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5540 - accuracy: 0.7268
Epoch 6/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5532 - accuracy: 0.7256
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5519 - accuracy: 0.7268
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5512 - accuracy: 0.7281
Epoch 9/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5507 - accuracy: 0.7291
Epoch 10/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5496 - accuracy: 0.7304
Epoch 11/5

In [241]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5533 - accuracy: 0.7292 - 427ms/epoch - 2ms/step
Loss: 0.553272008895874, Accuracy: 0.7292128205299377


In [242]:
# Export our model to HDF5 file
from keras.models import save_model


model_filename ='Export/AlphabetSoupCharity.h5'

# Save the trained model to an HDF5 file with additional options
save_model(nn, model_filename, save_format='h5')

print("Model saved to", model_filename)

Model saved to Export/AlphabetSoupCharity.h5
